# 0.4_matrix

In [58]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext blackcellmagic

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def nrows(n):
    pd.options.display.max_rows = n

nrows(5)

import itertools
from coding_the_matrix.Vec import Vec
from coding_the_matrix.mat import Mat
from coding_the_matrix.matutil import listlist2mat, value, keys, coldict2mat, rowdict2mat, mat2rowdict, mat2coldict
from coding_the_matrix.vecutil import *
from coding_the_matrix.GF2 import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [3]:
A = listlist2mat([[10, 20, 30, 40], [50, 60, 70, 80]])

In [4]:
print(A)

    0   1   2   3
0  10  20  30  40
1  50  60  70  80


In [5]:
def mat2Vec(M):
    R, C = M.D
    return Vec({(r,c) for r in R for c in C}, M.f) 

In [6]:
def tranpose(M):
    R, C = M.D
    D = (C, R)
    f = {(c, r): v for (r, c), v in M.f.items()}
    return Mat(D, f)

In [7]:
print(tranpose(A))

    0   1
0  10  50
1  20  60
2  30  70
3  40  80


In [8]:
def button_vectors(n):
    D = {(x, y) for x, y in itertools.product(range(n), range(n))}
    vecdict = {
        (i, j): Vec(
            D,
            dict([((x, j), one) for x in range(max(i - 1, 0), min(i + 2, n))]+
            [((i, y), one) for y in range(max(j - 1, 0), min(j + 2, n))]),
        )
        for (i, j) in D
    }
    return vecdict

In [9]:
vectors = button_vectors(5)

In [10]:
value(vectors)

Vec({(1, 3), (3, 0), (0, 2), (2, 1), (0, 3), (4, 0), (1, 2), (3, 3), (4, 4), (2, 2), (0, 4), (4, 1), (1, 1), (3, 2), (0, 0), (1, 4), (2, 3), (4, 2), (1, 0), (0, 1), (3, 1), (2, 0), (4, 3), (3, 4), (2, 4)}, {(0, 3): 1, (1, 3): 1, (2, 3): 1, (1, 2): 1, (1, 4): 1})

In [11]:
keys(vectors)

dict_keys([(1, 3), (3, 0), (0, 2), (2, 1), (0, 3), (4, 0), (1, 2), (3, 3), (4, 4), (2, 2), (0, 4), (4, 1), (1, 1), (3, 2), (0, 0), (1, 4), (2, 3), (4, 2), (1, 0), (0, 1), (3, 1), (2, 0), (4, 3), (3, 4), (2, 4)])

In [12]:
len(vectors)

25

In [13]:
b = vectors[(0, 0)]

In [14]:
dict([(4, 3)] + [(4, 1)])

{4: 1}

In [15]:
B = coldict2mat(button_vectors(5))

In [16]:
s = Vec(b.D, {(2, 2): one})

## Gnome

In [17]:
D = {'metal', 'concrete', 'plastic', 'water', 'electricity'}
v_gnome = Vec(D, {'concrete': 1.3, 'plastic': .2, 'water': .8, 'electricity': .4})
v_hoop = Vec(D, {'plastic': 1.5, 'water': .4, 'electricity': .3})
v_slinky = Vec(D, {'metal': .25, 'water': .2, 'electricity': .7})
v_putty = Vec(D, {'plastic': .3, 'water': .7, 'electricity': .5})
v_shooter = Vec(D, {'metal': .15, 'plastic': .5, 'water': .4, 'electricity': .8})

In [18]:
240*v_gnome + 55 * v_hoop

Vec({'metal', 'concrete', 'water', 'electricity', 'plastic'}, {'electricity': 112.5, 'water': 214.0, 'concrete': 312.0, 'plastic': 130.5})

In [88]:
rowdict = {
    'gnome': v_gnome,
    'hoop': v_hoop,
    'slinky': v_slinky,
    'putty': v_putty,
    'shooter': v_shooter,
}

In [89]:
M = rowdict2mat(rowdict)

In [90]:
print(M)

         metal  concrete  water  electricity  plastic
shooter   0.15       0.0    0.4          0.8      0.5
hoop      0.00       0.0    0.4          0.3      1.5
putty     0.00       0.0    0.7          0.5      0.3
slinky    0.25       0.0    0.2          0.7      0.0
gnome     0.00       1.3    0.8          0.4      0.2


In [22]:
R = set(rowdict.keys())

In [23]:
C = M.D[1]

In [24]:
u = Vec(R, {'putty': 133, 'gnome': 240, 'slinky': 150, 'hoop': 55, 'shooter': 90})

In [25]:
M.D

({'gnome', 'hoop', 'putty', 'shooter', 'slinky'},
 {'concrete', 'electricity', 'metal', 'plastic', 'water'})

Vec(M.D[1], {k: u * vec for k, vec in mat2coldict(M).items()})

In [26]:
u.D

{'gnome', 'hoop', 'putty', 'shooter', 'slinky'}

In [27]:
isinstance(M, Mat)

True

In [28]:
u * M

Vec({'metal', 'concrete', 'water', 'electricity', 'plastic'}, {'metal': 51.0, 'concrete': 312.0, 'water': 373.1, 'electricity': 356.0, 'plastic': 215.4})

In [29]:
A.D

({0, 1}, {0, 1, 2, 3})

In [40]:
sorted(A.f.keys())

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3)]

In [46]:
{k: A.f[k] for k in sorted(A.f)}

{(0, 0): 10,
 (0, 1): 20,
 (0, 2): 30,
 (0, 3): 40,
 (1, 0): 50,
 (1, 1): 60,
 (1, 2): 70,
 (1, 3): 80}

In [42]:
sorted([(1, 0), (0, 0), (0, 1)])

[(0, 0), (0, 1), (1, 0)]

In [41]:
A.f.keys()

dict_keys([(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3)])

In [37]:
import sys
sys.path.append('../coding_the_matrix/mat')

In [83]:
from coding_the_matrix import solver

In [91]:
solver.solve()

TypeError: solve() missing 2 required positional arguments: 'A' and 'b'

In [94]:
b = Vec({'metal', 'concrete', 'water', 'electricity', 'plastic'}, {'metal': 51.0, 'concrete': 312.0, 'water': 373.1, 'electricity': 356.0, 'plastic': 215.4})

In [95]:
M.transpose()

Mat(({'metal', 'concrete', 'water', 'electricity', 'plastic'}, {'shooter', 'hoop', 'putty', 'slinky', 'gnome'}), {('concrete', 'gnome'): 1.3, ('concrete', 'hoop'): 0, ('concrete', 'putty'): 0, ('concrete', 'shooter'): 0, ('concrete', 'slinky'): 0, ('electricity', 'gnome'): 0.4, ('electricity', 'hoop'): 0.3, ('electricity', 'putty'): 0.5, ('electricity', 'shooter'): 0.8, ('electricity', 'slinky'): 0.7, ('metal', 'gnome'): 0, ('metal', 'hoop'): 0, ('metal', 'putty'): 0, ('metal', 'shooter'): 0.15, ('metal', 'slinky'): 0.25, ('plastic', 'gnome'): 0.2, ('plastic', 'hoop'): 1.5, ('plastic', 'putty'): 0.3, ('plastic', 'shooter'): 0.5, ('plastic', 'slinky'): 0, ('water', 'gnome'): 0.8, ('water', 'hoop'): 0.4, ('water', 'putty'): 0.7, ('water', 'shooter'): 0.4, ('water', 'slinky'): 0.2})

In [96]:
solution = solver.solve(M.transpose(), b)

AssertionError: 

In [113]:
import dis

In [125]:
A = M.transpose()

In [126]:
A

Mat(({'metal', 'concrete', 'water', 'electricity', 'plastic'}, {'shooter', 'hoop', 'putty', 'slinky', 'gnome'}), {('concrete', 'gnome'): 1.3, ('concrete', 'hoop'): 0, ('concrete', 'putty'): 0, ('concrete', 'shooter'): 0, ('concrete', 'slinky'): 0, ('electricity', 'gnome'): 0.4, ('electricity', 'hoop'): 0.3, ('electricity', 'putty'): 0.5, ('electricity', 'shooter'): 0.8, ('electricity', 'slinky'): 0.7, ('metal', 'gnome'): 0, ('metal', 'hoop'): 0, ('metal', 'putty'): 0, ('metal', 'shooter'): 0.15, ('metal', 'slinky'): 0.25, ('plastic', 'gnome'): 0.2, ('plastic', 'hoop'): 1.5, ('plastic', 'putty'): 0.3, ('plastic', 'shooter'): 0.5, ('plastic', 'slinky'): 0, ('water', 'gnome'): 0.8, ('water', 'hoop'): 0.4, ('water', 'putty'): 0.7, ('water', 'shooter'): 0.4, ('water', 'slinky'): 0.2})

In [122]:
b

Vec({'metal', 'concrete', 'water', 'electricity', 'plastic'}, {'metal': 51.0, 'concrete': 312.0, 'water': 373.1, 'electricity': 356.0, 'plastic': 215.4})

In [134]:
rowdict = mat2rowdict(A)

In [135]:
rowdict

{'concrete': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0, 'hoop': 0, 'putty': 0, 'slinky': 0, 'gnome': 1.3}),
 'electricity': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.8, 'hoop': 0.3, 'putty': 0.5, 'slinky': 0.7, 'gnome': 0.4}),
 'metal': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.15, 'hoop': 0, 'putty': 0, 'slinky': 0.25, 'gnome': 0}),
 'plastic': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.5, 'hoop': 1.5, 'putty': 0.3, 'slinky': 0, 'gnome': 0.2}),
 'water': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.4, 'hoop': 0.4, 'putty': 0.7, 'slinky': 0.2, 'gnome': 0.8})}

In [130]:
b

Vec({'metal', 'concrete', 'water', 'electricity', 'plastic'}, {'metal': 51.0, 'concrete': 312.0, 'water': 373.1, 'electricity': 356.0, 'plastic': 215.4})

In [133]:
b.D

{'concrete', 'electricity', 'metal', 'plastic', 'water'}

In [183]:
R, C = A.D
assert len(R) == len(C)
N = len(R)

In [184]:
R

{'concrete', 'electricity', 'metal', 'plastic', 'water'}

In [186]:
b

Vec({'metal', 'concrete', 'water', 'electricity', 'plastic'}, {'metal': 51.0, 'concrete': 312.0, 'water': 373.1, 'electricity': 356.0, 'plastic': 215.4})

In [ ]:
from typing import Tuple
def get_max_row(col, rowdict) -> Tuple[str, Vec]:
    """get row from rowdict where col is max"""
    return sorted(rowdict.items(), key= lambda x: x[1].f[col])[-1]

In [243]:
rowdict = mat2rowdict(A)
upper_triangle = {}  # 結果をいれる

# select col to operate on
for col in C:
    # get row where col is max value
    row, rowvec = get_max_row(col, rowdict)
    rowdict.pop(row)
    upper_triangle.update({row: rowvec}.copy())
    
    # iterate over the rows
    for other, othervec in rowdict.items():
        # coef
        coef = othervec[col]/rowvec[col]

        # update rowvec
        othervec -= coef * rowvec
        rowdict[other] = othervec

        # update b
        b[other] -= coef * b[row]

In [244]:
upper_triangle

{'electricity': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.8, 'hoop': 0.3, 'putty': 0.5, 'slinky': 0.7, 'gnome': 0.4}),
 'plastic': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.0, 'hoop': 1.3125, 'putty': -0.012500000000000011, 'slinky': -0.4375, 'gnome': -0.04999999999999999}),
 'water': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.0, 'hoop': 0.0, 'putty': 0.45238095238095233, 'slinky': -0.06666666666666664, 'gnome': 0.6095238095238096}),
 'metal': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.0, 'hoop': 6.938893903907228e-18, 'putty': 0.0, 'slinky': 0.08610526315789478, 'gnome': 0.04989473684210528}),
 'concrete': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.0, 'hoop': 0.0, 'putty': 0.0, 'slinky': 0.0, 'gnome': 1.3})}

In [250]:
M2 = rowdict2mat(upper_triangle)

In [251]:
print(M2)

             shooter          hoop     putty    slinky     gnome
metal            0.0  6.938894e-18  0.000000  0.086105  0.049895
concrete         0.0  0.000000e+00  0.000000  0.000000  1.300000
water            0.0  0.000000e+00  0.452381 -0.066667  0.609524
electricity      0.8  3.000000e-01  0.500000  0.700000  0.400000
plastic          0.0  1.312500e+00 -0.012500 -0.437500 -0.050000


In [210]:
# coef
coef = othervec[col]/rowvec[col]

# update rowvec
othervec -= coef * rowvec
rowdict[other] = othervec

# update b
b[other] -= coef * b[row]

In [213]:
b

Vec({'metal', 'concrete', 'water', 'electricity', 'plastic'}, {'metal': 51.0, 'concrete': 312.0, 'water': 373.1, 'electricity': 356.0, 'plastic': 215.4})

In [220]:
row, b[row], other, b[other]

('electricity', 356.0, 'metal', 51.0)

In [221]:
coef

0.0

In [219]:
print(A)

             shooter  hoop  putty  slinky  gnome
metal           0.15   0.0    0.0    0.25    0.0
concrete        0.00   0.0    0.0    0.00    1.3
water           0.40   0.4    0.7    0.20    0.8
electricity     0.80   0.3    0.5    0.70    0.4
plastic         0.50   1.5    0.3    0.00    0.2


In [217]:
col

'shooter'

In [211]:
other

'metal'

In [212]:
rowdict

{'metal': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.0, 'hoop': -0.05624999999999999, 'putty': -0.09374999999999999, 'slinky': 0.11875000000000002, 'gnome': -0.075}),
 'concrete': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0, 'hoop': 0, 'putty': 0, 'slinky': 0, 'gnome': 1.3}),
 'water': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.4, 'hoop': 0.4, 'putty': 0.7, 'slinky': 0.2, 'gnome': 0.8}),
 'plastic': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.5, 'hoop': 1.5, 'putty': 0.3, 'slinky': 0, 'gnome': 0.2})}

In [175]:
rowvec

Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0, 'hoop': 0, 'putty': 0, 'slinky': 0, 'gnome': 1.3})

In [173]:
othervec

Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.8, 'hoop': 0.3, 'putty': 0.5, 'slinky': 0.7, 'gnome': 0.4})

In [176]:
rowvec[col]

1.3

In [182]:
rowdict

{'concrete': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0, 'hoop': 0, 'putty': 0, 'slinky': 0, 'gnome': 1.3}),
 'electricity': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.8, 'hoop': 0.3, 'putty': 0.5, 'slinky': 0.7, 'gnome': 0.4}),
 'metal': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.15, 'hoop': 0, 'putty': 0, 'slinky': 0.25, 'gnome': 0}),
 'plastic': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.5, 'hoop': 1.5, 'putty': 0.3, 'slinky': 0, 'gnome': 0.2}),
 'water': Vec({'shooter', 'hoop', 'putty', 'slinky', 'gnome'}, {'shooter': 0.4, 'hoop': 0.4, 'putty': 0.7, 'slinky': 0.2, 'gnome': 0.8})}

In [ ]:
sol

In [ ]:
solver

In [105]:
solver.solve.__code__

<code object solve at 0x16835e150, file "../resources/private/solver.py", line 186>

In [106]:
import inspect

In [108]:
solver.solve?

Signature: solver.solve(A, b, eps=1e-15)
Docstring:
Solve the matrix-vector equation Ax = b.

If a solution to Ax = b does not exist, then the vector returned by
solve(A, b) is not a solution.  Please verify that Ax = b.

Args:
    A: A matrix of type Mat.
    b: A vector of type Vec.
    eps: A threshold.  Optional.

Returns:
    x: A vector of type Vec.

Raises:
    AssertionError: An error occurs when A is not a matrix of type Mat.
    AssertionError: An error occurs when b is not a vector of type Vec.
    AssertionError: An error occurs when A.D[0] != b.D.

Example 1: Solve Ax = b and verify that x is close to b.
>>> from vec import Vec
>>> from mat import Mat
>>> A = mat.Mat(({0, 1, 2}, {0, 1}), {(0, 1): 2, (2, 0): 10, (1, 0): 3, (0, 0): 1, (1, 1): 4})
>>> b = vec.Vec({0, 1, 2},{0: 1, 1: 5, 2: 30})
>>> A.D[0] == b.D
True
>>> x = solve(A, b)
>>> A.D[1] == x.D
True
>>> (b-A*x).is_almost_zero()
True

Example 2: Solve Ax = b and see that x is not a valid solution.
>>> A = mat.Mat(({0,

In [101]:
b

Vec({'metal', 'concrete', 'water', 'electricity', 'plastic'}, {'metal': 51.0, 'concrete': 312.0, 'water': 373.1, 'electricity': 356.0, 'plastic': 215.4})

In [102]:
M.transpose()

Mat(({'metal', 'concrete', 'water', 'electricity', 'plastic'}, {'shooter', 'hoop', 'putty', 'slinky', 'gnome'}), {('concrete', 'gnome'): 1.3, ('concrete', 'hoop'): 0, ('concrete', 'putty'): 0, ('concrete', 'shooter'): 0, ('concrete', 'slinky'): 0, ('electricity', 'gnome'): 0.4, ('electricity', 'hoop'): 0.3, ('electricity', 'putty'): 0.5, ('electricity', 'shooter'): 0.8, ('electricity', 'slinky'): 0.7, ('metal', 'gnome'): 0, ('metal', 'hoop'): 0, ('metal', 'putty'): 0, ('metal', 'shooter'): 0.15, ('metal', 'slinky'): 0.25, ('plastic', 'gnome'): 0.2, ('plastic', 'hoop'): 1.5, ('plastic', 'putty'): 0.3, ('plastic', 'shooter'): 0.5, ('plastic', 'slinky'): 0, ('water', 'gnome'): 0.8, ('water', 'hoop'): 0.4, ('water', 'putty'): 0.7, ('water', 'shooter'): 0.4, ('water', 'slinky'): 0.2})

Vec({'metal', 'concrete', 'water', 'electricity', 'plastic'}, {'metal': 51.0, 'concrete': 312.0, 'water': 373.1, 'electricity': 356.0, 'plastic': 215.4})

In [ ]:
solution

In [ ]:
residual = b - solution * M

In [ ]:
assert residual * residual < 1e-10

In [49]:
print(M)

         metal  concrete  water  electricity  plastic
hoop      0.00       0.0    0.4          0.3      1.5
gnome     0.00       1.3    0.8          0.4      0.2
putty     0.00       0.0    0.7          0.5      0.3
slinky    0.25       0.0    0.2          0.7      0.0
shooter   0.15       0.0    0.4          0.8      0.5


In [ ]:
M.tra

In [50]:
M = coldict2mat({0:Vec({0,1},{0:1,1:2}),1:Vec({0,1},{0:3,1:4})})

In [54]:
b = Vec({0, 1}, {0: 5, 1: 6})

In [57]:
b

Vec({0, 1}, {0: 5, 1: 6})

In [53]:
M.D

({0, 1}, {0, 1})

In [52]:
print(M)

   0  1
0  1  3
1  2  4


In [61]:
rowdict = mat2rowdict(M)

In [62]:
rowdict

{0: Vec({0, 1}, {0: 1, 1: 3}), 1: Vec({0, 1}, {0: 2, 1: 4})}

In [78]:
R = sorted(M.D[0])
C = sorted(M.D[1])

In [80]:
C

[0, 1]

In [79]:
vec = rowdict[R[0]]

In [74]:
vec

Vec({0, 1}, {0: 1, 1: 3})

In [ ]:
# get a vec
# get the domain of the vec
# get a 

In [14]:
import itertools
for r, c in itertools.product(*D):
    print(f"('{r}', '{c}'): ,")

('a', '@'): ,
('a', '#'): ,
('a', '?'): ,
('b', '@'): ,
('b', '#'): ,
('b', '?'): ,


In [7]:
D = ({'a', 'b'}, {'@', '#', '?'})

In [12]:
f = {
    ("a", "@"): 1,
    ("a", "#"): 2,
    ("a", "?"): 3,
    ("b", "@"): 10,
    ("b", "#"): 20,
    ("b", "?"): 30,
}

In [13]:
M = Mat(D, f)

In [31]:
def mat2rowdict(M):
    """return the rowdict representation of M"""
    R, C = M.D
    return {r: Vec(C, {c: M.f[(r, c)] for c in C}) for r in R}

In [32]:
mat2rowdict(M)

{'a': Vec({'@', '#', '?'}, {'@': 1, '#': 2, '?': 3}),
 'b': Vec({'@', '#', '?'}, {'@': 10, '#': 20, '?': 30})}

In [33]:
def mat2coldict(M):
    """return the coldict representation of M"""
    R, C = M.D
    return {c: Vec(R, {r: M.f[(r, c)] for r in R}) for c in C}

In [34]:
mat2coldict(M)

{'@': Vec({'a', 'b'}, {'a': 1, 'b': 10}),
 '#': Vec({'a', 'b'}, {'a': 2, 'b': 20}),
 '?': Vec({'a', 'b'}, {'a': 3, 'b': 30})}